In [1]:
import importlib
import copy
import pickle
# --------------------
from sklearn.utils import gen_batches, check_array
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.utils.validation import FLOAT_DTYPES
import numpy as np
from numpy.linalg import norm,inv,matrix_rank, pinv
# --------------------
from skopt.space import Integer
from skopt.utils import use_named_args
from skopt import gp_minimize
# --------------------
import h5py
from sklearn.cross_decomposition import PLSRegression

# import warnings
# warnings.filterwarnings('ignore')

from pytictoc import TicToc
tim=TicToc()
tim_tot = TicToc()

In [2]:
def RollingCV(tscv,X):
    cv = tscv.split(X)
    (train_index, test_index) = next(cv)
    yield(
        train_index,
        test_index
    )
    test_size = len(test_index)

    for (train_index, test_index) in (cv):
        yield(
            train_index[-test_size:],
            test_index
        )

In [3]:
with h5py.File('./data/merra2_t.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)

    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]

n_fold = 74
test_size=30    

4412


In [ ]:
with h5py.File('./data/TW_PM25.h5', 'r') as f:
    X_train, Y_train = f['X_train'], f['Y_train']
    n_train = X_train.shape[0]
    print(n_train)
        
    X_train = X_train[0:n_train]
    Y_train = Y_train[0:n_train]
n_fold = 17
test_size=30

In [4]:
def dump_PLS(PLS, fn):
    np.savez(fn,
             n=PLS.n,
             n_comp=PLS.n_components,
             W = PLS.W,
             y_loadings = np.dot(PLS.S.T, PLS.W),
             CW = np.dot(PLS.C, PLS.W),
             x_mean = PLS._x_mean,
             y_mean = PLS._y_mean
             )

def predict(dat, X, n_comp):
    X = check_array(X, copy=copy, dtype=FLOAT_DTYPES)
    W = dat['W'][:,:n_comp]
    CW = dat['CW'][:,:n_comp]
    y_loadings = dat['y_loadings'][:,:n_comp]

    x_rotaions = np.dot(
        W,
        pinv(np.dot(W.T,
                    CW))
    )

    coef = np.dot(x_rotaions, y_loadings.T)
    X -= dat['x_mean']
    ypred = np.dot(X, coef)
    ypred += dat['y_mean']

    return ypred


In [5]:
print(X_train.shape)
print(Y_train.shape)

tscv = TimeSeriesSplit(n_splits=n_fold, test_size=test_size)
for i, (train_index, test_index) in enumerate(tscv.split(X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

print("="*40)
for i, (train_index, test_index) in enumerate(RollingCV(tscv,X_train)):
    print(f"Fold {i}:")
    print(f"  Train: len={len(train_index)}")
    print(f"  Test:  len={len(test_index)}")
    if(i>3):
        print(f"  Train: {train_index}")
        print(f"  Test:  {test_index}")
        break

(4412, 30456)
(4412, 30456)
Fold 0:
  Train: len=2192
  Test:  len=30
Fold 1:
  Train: len=2222
  Test:  len=30
Fold 2:
  Train: len=2252
  Test:  len=30
Fold 3:
  Train: len=2282
  Test:  len=30
Fold 4:
  Train: len=2312
  Test:  len=30
  Train: [   0    1    2 ... 2309 2310 2311]
  Test:  [2312 2313 2314 2315 2316 2317 2318 2319 2320 2321 2322 2323 2324 2325
 2326 2327 2328 2329 2330 2331 2332 2333 2334 2335 2336 2337 2338 2339
 2340 2341]
Fold 0:
  Train: len=2192
  Test:  len=30
Fold 1:
  Train: len=30
  Test:  len=30
Fold 2:
  Train: len=30
  Test:  len=30
Fold 3:
  Train: len=30
  Test:  len=30
Fold 4:
  Train: len=30
  Test:  len=30
  Train: [2282 2283 2284 2285 2286 2287 2288 2289 2290 2291 2292 2293 2294 2295
 2296 2297 2298 2299 2300 2301 2302 2303 2304 2305 2306 2307 2308 2309
 2310 2311]
  Test:  [2312 2313 2314 2315 2316 2317 2318 2319 2320 2321 2322 2323 2324 2325
 2326 2327 2328 2329 2330 2331 2332 2333 2334 2335 2336 2337 2338 2339
 2340 2341]


In [ ]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

n_comp_list=list(range(1,21))
print("n_comp:", n_comp_list)
score_all = np.zeros((n_fold, len(n_comp_list)))

n_comp_max=20
PLS = ISIMPLS(n_components=n_comp_max)

tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    y_true = Y_train[test_index]
    for j,n_comp in enumerate(n_comp_list):
        y_pred = PLS.predict(X_train[test_index], n_comp)
        score_all[i,j] = RMSE(y_true, y_pred)
    if (i%5==0): 
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()

scores = score_all.mean(axis=0)
best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {n_comp_list[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")
print(scores)

In [6]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

n_comp_max=100
PLS = ISIMPLS(n_components=n_comp_max)
tim_tot.tic()
tim.tic()
for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
    PLS.fit(X_train[train_index], Y_train[train_index])
    dump_PLS(PLS, f'./data/wrk/ISIMPLS_fold{i:02d}.npz')
    if (i%5==0):
        print(f"fold: {i+1:02d}, elapsed time: {tim.tocvalue():.1f}s")
        tim.tic()
print(f"total time={tim_tot.tocvalue():.1f}s")

fold: 01, elapsed time: 1896.0s


fold: 06, elapsed time: 8730.3s


fold: 11, elapsed time: 7269.3s


fold: 16, elapsed time: 7354.7s


fold: 21, elapsed time: 7604.7s


...

fold: 56, elapsed time: 7203.6s


fold: 61, elapsed time: 6216.3s


fold: 66, elapsed time: 6816.0s


fold: 71, elapsed time: 6431.9s


total time=109200.4s


In [7]:
PLS_list = []
for i in range(n_fold):
    PLS_list.append(np.load(f'./data/wrk/ISIMPLS_fold{i:02d}.npz'))


In [8]:
space  = [Integer(1, n_comp_max, name='n_components')]
@use_named_args(space)
def Comp_Model_Score(n_components):
    scores = np.zeros((n_fold,))
    for i,(train_index, test_index) in enumerate(RollingCV(tscv, X_train)):
        y_true = Y_train[test_index]
        y_pred = predict(PLS_list[i], X_train[test_index], n_components)
        # y_pred = predict(np.load(f'./data/wrk/ISIMPLS_fold{i:02d}.npz'),
        #                  X_train[test_index], n_components)
        scores[i] = RMSE(y_true, y_pred)
    return np.mean(scores)


In [9]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")
n_calls *= 2

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=max(n_calls,10), 
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best param = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


n_calls = 5
Iteration No: 1 started. Evaluating function at random point.


Iteration No: 1 ended. Evaluation done at random point.
Time taken: 198.1572
Function value obtained: 2.1518
Current minimum: 2.1518
Iteration No: 2 started. Evaluating function at random point.


Iteration No: 2 ended. Evaluation done at random point.
Time taken: 198.8516
Function value obtained: 2.1548
Current minimum: 2.1518
Iteration No: 3 started. Evaluating function at random point.


Iteration No: 3 ended. Evaluation done at random point.
Time taken: 198.9420
Function value obtained: 2.1561
Current minimum: 2.1518
Iteration No: 4 started. Evaluating function at random point.


Iteration No: 4 ended. Evaluation done at random point.
Time taken: 198.4083
Function value obtained: 2.1548
Current minimum: 2.1518
Iteration No: 5 started. Evaluating function at random point.


...

Iteration No: 6 ended. Evaluation done at random point.
Time taken: 193.6934
Function value obtained: 2.1936
Current minimum: 2.1495
Iteration No: 7 started. Evaluating function at random point.


Iteration No: 7 ended. Evaluation done at random point.
Time taken: 194.3763
Function value obtained: 2.2463
Current minimum: 2.1495
Iteration No: 8 started. Evaluating function at random point.


Iteration No: 8 ended. Evaluation done at random point.
Time taken: 193.4198
Function value obtained: 2.9291
Current minimum: 2.1495
Iteration No: 9 started. Evaluating function at random point.


Iteration No: 9 ended. Evaluation done at random point.
Time taken: 214.8643
Function value obtained: 2.2605
Current minimum: 2.1495
Iteration No: 10 started. Evaluating function at random point.


Iteration No: 10 ended. Evaluation done at random point.
Time taken: 215.6045
Function value obtained: 2.1711
Current minimum: 2.1495
----------------------------------------
Best param = 63
Best score = 2.1495
total time = 2002.3s


In [ ]:
n_calls = int(np.log(n_comp_max)) + 1
print(f"n_calls = {n_calls:d}")
n_calls *= 3

tim_tot.tic()
res_gp = gp_minimize(Comp_Model_Score, space, n_calls=max(n_calls,10), 
                     random_state=0, verbose=True)

print("-"*40)
print(f"Best param = {res_gp.x[0]:02d}")
print(f"Best score = {res_gp.fun:.4f}")
print(f"total time = {tim_tot.tocvalue():.1f}s")


In [ ]:
import Code.ISIMPLS
importlib.reload(Code.ISIMPLS)
from Code.ISIMPLS import ISIMPLS2 as ISIMPLS

scores = []
params = []
n_comp_list=list(range(100,401,50))

tim_tot.tic()
for n_comp in n_comp_list:
    tim.tic()
    pls=ISIMPLS(n_components=n_comp)
    scores.append(Comp_Model_Score(pls, RollingCV(tscv,X_train), X_train, Y_train) )
    params.append({'n_components': n_comp})
    print(f"params={params[-1]}, score={scores[-1]:.7e}, "
          +f"elapsed time={tim.tocvalue():.1f}s")

best_ind = np.nanargmin(scores)
print("")
print(f"best parameter: {params[best_ind]}; score: {scores[best_ind]:.7e}; "
      + f"total time={tim_tot.tocvalue():.1f}s")


params={'n_components': 100}, score=2.1730468e+00, elapsed time=97598.1s


params={'n_components': 150}, score=2.2380231e+00, elapsed time=159673.6s
